In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import pytz

API_KEY = "492aa1d288cfdf972a104e0a55c35b68"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

city_list = ["Delhi","Mumbai","Bangalore","Chennai","Kolkata","Hyderabad","Pune",
             "Jaipur","Lucknow","Surat","Kanpur","Nagpur","Indore","Bhopal",
             "Patna","Vadodara","Ludhiana","Agra","Varanasi"]

india_tz = pytz.timezone("Asia/Kolkata")

def to_ist(unix_utc):
    if unix_utc is None:
        return None
    return datetime.fromtimestamp(unix_utc, tz=timezone.utc).astimezone(india_tz).strftime("%Y-%m-%d %H:%M:%S")

def to_local_with_offset(unix_utc, offset_seconds):
    """Convert UNIX UTC seconds to local city time using the API's timezone offset (+/- seconds)."""
    if unix_utc is None:
        return None
    dt_local = datetime.utcfromtimestamp(unix_utc) + timedelta(seconds=(offset_seconds or 0))
    return dt_local.strftime("%Y-%m-%d %H:%M:%S")

all_weather_data = []

for city in city_list:
    params = {"q": city, "appid": API_KEY, "units": "metric"}
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    if response.status_code == 200:
        sys_block = data.get("sys", {}) or {}
        tz_offset = data.get("timezone")  
        sunrise_unix = sys_block.get("sunrise")
        sunset_unix  = sys_block.get("sunset")

        weather_info = {
            "date_time": datetime.now(india_tz).strftime("%Y-%m-%d %H:%M"),
            "city": city,
            "temperature(°C)": data["main"]["temp"],
            "feels_like(°C)": data["main"]["feels_like"],
            "temp_min(°C)": data["main"]["temp_min"],
            "temp_max(°C)": data["main"]["temp_max"],
            "pressure(hPa)": data["main"]["pressure"],
            "humidity(%)": data["main"]["humidity"],
            "visibility(m)": data.get("visibility"),
            "wind_speed(m/s)": data["wind"]["speed"],
            "wind_deg(°)": data["wind"].get("deg"),
            "cloudiness(%)": data["clouds"]["all"],
            "weather_main": data["weather"][0]["main"],
            "weather_description": data["weather"][0]["description"],
            "sunrise_ist": to_ist(sunrise_unix),
            "sunset_ist":  to_ist(sunset_unix)
        }
        all_weather_data.append(weather_info)
    else:
        print(f"Failed for {city}: {data}")

df_weather = pd.DataFrame(all_weather_data)
display(df_weather)


StatementMeta(, d30c4ce3-422f-4d80-b30a-49191c19fc7a, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 625a0509-2839-4ec1-b8b9-f14dbe014487)

In [2]:
import pandas as pd
from datetime import datetime

india_tz = pytz.timezone('Asia/Kolkata')
today_str = datetime.now(india_tz).strftime("%Y-%m-%d %H:%M:%S")

file_path = f"/lakehouse/default/Files/weather_data_bronze/weather_data {today_str}.csv"
df_weather.to_csv(file_path, index=False)

print(f"Saved: {file_path}")

StatementMeta(, 6871bbce-14ab-4d8e-8eb3-d718b6182ed7, 4, Finished, Available, Finished)

Saved: /lakehouse/default/Files/weather_data_bronze/weather_data 2025-08-30 06:19:28.csv
